In [43]:
import sys
sys.path.append('../../../../laddie/src/')
sys.path.append('../../../src2/')

import numpy as np
import xarray as xr
import pandas as pd
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo

from laddie import Laddie
from geometry3 import Geometry
from forcing import Forcing
from tools import im,jm
from tools import im,jm


%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
exp_class = os.getcwd()[64:-5]
run = os.getcwd()[-4:]
savename = f'{exp_class}_{run}'
print(f'Documents are saved to folder \'{os.getcwd()}\' with save name', savename)

Documents are saved to folder '/Users/5941962/Documents/GitHub/laddie-run-analysis/experiments/Ocean3/r001' with save name Ocean3_r001


In [45]:
# Select input file geometry and initial geometry at t = 0
ds = xr.open_dataset('../../../data/OC3_prepped_boundary.nc')
ds_t0 = ds.isel(t=0)

# Generate geometry
N = 2                          # standard resolution is at 500 m, N = 2: 1 km, N = 4: 2 km !!! DOES NOT WORK PROPERLY
geom = Geometry(ds_t0, savename)
geom.coarsen(N=N)
geom = geom.create()

# Select forcing
forc = Forcing(geom).isomip('WARM')

Resolution set to 1.0 km
Geometry Ocean3_r001_1.0 created


In [47]:
# Set up LADDIE
laddie = Laddie(forc)

laddie.Ah = 6               #Horizontal viscosity
laddie.Kh = 1               #Horizontal diffusivity
laddie.ci = 0               #Assume insulation ice
laddie.slip = .5            #Semi-slip
laddie.convop = 1

laddie.correctisf = True    #Correct ice front
laddie.nnUV = 1             #Weight of nearest neighbours for new cells
laddie.vcut = 1             #maximum velocity

laddie.Cdtop = 2.5e-3       #Top drag coefficient
laddie.minD = 4             #Minimum layer thickness
laddie.dt = 120             #Time step
laddie.nu = .5
laddie.gamTfix = 0.00018

In [48]:
laddiedays = 1 # number of days to integrate laddie for each ice time step
laddiespinup = 20 # number of days to spin up laddie

laddie.saveday = laddiedays
laddie.restday = 100

ds = laddie.compute(days=laddiespinup, savename=f'{os.getcwd()}/{savename}')

Starting from noflow
   0.100 days ||   9.1  [4.91   12] m ||  42.89 |  114 m/yr || -5.92 % || 0.269 + 0.000 - 0.549 |  0.536 Sv ||  0.23 m/s ||  0.06555   0 []
   0.200 days ||   8.8  [4.00   23] m ||  37.86 |  98 m/yr ||  5.70 % || 0.407 + 0.018 - 0.197 |  0.417 Sv ||  0.26 m/s ||  0.02226   0 []
   0.300 days ||   8.8  [4.00   38] m ||  33.75 |  95 m/yr ||  3.96 % || 0.380 + 0.021 - 0.102 |  0.311 Sv ||  0.26 m/s ||  0.02047   0 []
   0.400 days ||   8.8  [4.00   52] m ||  30.29 |  94 m/yr ||  3.93 % || 0.322 + 0.021 - 0.074 |  0.267 Sv ||  0.25 m/s ||  0.03287   0 []
   0.500 days ||   8.8  [4.00   59] m ||  27.36 |  94 m/yr ||  3.31 % || 0.324 + 0.023 - 0.056 |  0.259 Sv ||  0.25 m/s ||  0.03449   0 []
   0.600 days ||   8.8  [4.00   62] m ||  24.97 |  93 m/yr ||  2.44 % || 0.375 + 0.027 - 0.038 |  0.269 Sv ||  0.25 m/s ||  0.03577   0 []
   0.700 days ||   8.9  [4.00   65] m ||  23.08 |  92 m/yr ||  1.88 % || 0.432 + 0.031 - 0.025 |  0.283 Sv ||  0.25 m/s ||  0.03857   0 []
   0.